### Connecting to a SQL Database Project

In [4]:
import os
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

#### Part 1: Install Dependencies

Run the command pip install -r requirements.txt

#### Part 2: Add Your Database Credentials

DB_USER='remelop'
DB_PASSWORD='Melo0316'
DB_PORT= 3306
DB_HOST='localhost'
DB_NAME='exercise'

#### Part 3 : Create a Database Using PostgreSQL

Make sure you have installed the Postgres client for the terminal called PSQL by running the following command:

$ psql --version

1. Postgres is a database server, you have to start the server before you can use it. In order to stat postgres in your computer run the following command:

sudo service postgresql start

2. Create a new user to connect to your database:

psql -U postgres -c "CREATE USER remelop;"

3. Create a new database within the Postgres engine by customizing and executing the following command:

psql -U postgres -c "CREATE DATABASE exercise OWNER remelop;"

4. Connect to the Postgres engine to use your database, manipulate tables and data:

psql -h localhost -U remelop exercise

#### Part 4: Understand the structure of your project's template

Once you have completed the above steps, you will have created your SQL database, but there are no tables yet. Next, you need to connect to your empty database, and you will create some tables using Python via SQL scripts that you will find in the ./src/sql/ folder.

#### Part 5: Start programming

1) Connect to the database with SQLAlchemy

In [5]:
# Load environment variables
load_dotenv()

True

The connect function contains the code needed to connect to your Python database. If you look closely, you will see how it loads all the environment variables into a variable called connection_string and then calls the create_engine and connect functions.

In [8]:
def connect():
    global engine
    try:
        connection_string = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}/{os.getenv('DB_NAME')}"
        print("Starting the connection...")
        engine = create_engine(connection_string, isolation_level="AUTOCOMMIT")
        engine.connect()
        print("Connected successfully!")
        return engine
    except Exception as e:
        print(f"Error connecting to the database: {e}")
        return None

In [9]:
engine = connect()

#if engine is None:
#    exit() 


Starting the connection...
Connected successfully!


2) Create the tables

In [10]:
with engine.connect() as connection:
    connection.execute(text("""
    CREATE TABLE IF NOT EXISTS publishers (
        publisher_id SERIAL PRIMARY KEY,
        name VARCHAR(255) NOT NULL
    );

    CREATE TABLE IF NOT EXISTS authors (
        author_id SERIAL PRIMARY KEY,
        first_name VARCHAR(100) NOT NULL,
        middle_name VARCHAR(50) NULL,
        last_name VARCHAR(100) NULL
    );

    CREATE TABLE IF NOT EXISTS books (
        book_id SERIAL PRIMARY KEY,
        title VARCHAR(255) NOT NULL,
        total_pages INT NULL,
        rating DECIMAL(4, 2) NULL,
        isbn VARCHAR(13) NULL,
        published_date DATE,
        publisher_id INT NULL,
        CONSTRAINT fk_publisher FOREIGN KEY (publisher_id) REFERENCES publishers(publisher_id) ON DELETE SET NULL
    );

    CREATE TABLE IF NOT EXISTS book_authors (
        book_id INT NOT NULL,
        author_id INT NOT NULL,
        PRIMARY KEY (book_id, author_id),
        CONSTRAINT fk_book FOREIGN KEY (book_id) REFERENCES books(book_id) ON DELETE CASCADE,
        CONSTRAINT fk_author FOREIGN KEY (author_id) REFERENCES authors(author_id) ON DELETE CASCADE
    );
    """))

3) Insert data

In [11]:
with engine.connect() as connection:
    connection.execute(text("""
    INSERT INTO publishers (publisher_id, name) VALUES
        (1, 'O Reilly Media'),
        (2, 'A Book Apart'),
        (3, 'A K PETERS'),
        (4, 'Academic Press'),
        (5, 'Addison Wesley'),
        (6, 'Albert&Sweigart'),
        (7, 'Alfred A. Knopf')
    ON CONFLICT (publisher_id) DO NOTHING;

    INSERT INTO authors (author_id, first_name, middle_name, last_name) VALUES
        (1, 'Merritt', NULL, 'Eric'),
        (2, 'Linda', NULL, 'Mui'),
        (3, 'Alecos', NULL, 'Papadatos'),
        (4, 'Anthony', NULL, 'Molinaro'),
        (5, 'David', NULL, 'Cronin'),
        (6, 'Richard', NULL, 'Blum'),
        (7, 'Yuval', 'Noah', 'Harari'),
        (8, 'Paul', NULL, 'Albitz')
    ON CONFLICT (author_id) DO NOTHING;
    """))

4) Use Pandas to read and display a table

In [12]:
df = pd.read_sql("SELECT * FROM publishers;", engine)
print(df)

   publisher_id             name
0             1   O Reilly Media
1             2     A Book Apart
2             3       A K PETERS
3             4   Academic Press
4             5   Addison Wesley
5             6  Albert&Sweigart
6             7  Alfred A. Knopf
